### Importing of Packages

In [1]:
%%capture
!pip install ipython-autotime
%load_ext autotime

time: 15 ms (started: 2022-06-25 22:27:38 +01:00)


In [2]:
!pip install imblearn --user

time: 6.99 s (started: 2022-06-25 22:27:39 +01:00)


In [3]:
# Standard libraries
# Standard libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
import re
from string import punctuation
import nltk
nltk.download(['stopwords','punkt'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer 


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.utils import resample
from imblearn.pipeline import Pipeline
# Building classification models
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import xgboost as xgb
from sklearn.ensemble import StackingClassifier

# Model evaluation
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn import metrics

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMSON\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SAMSON\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


time: 20.4 s (started: 2022-06-25 22:27:46 +01:00)


### Loading of Dataset and general overview

In [4]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

time: 578 ms (started: 2022-06-25 22:28:06 +01:00)


In [5]:
# Splitting the labels and features
train.head(5)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


time: 93 ms (started: 2022-06-25 22:28:07 +01:00)


In [6]:
#Taking general overview at both datasets
print('TRAINING DATA')
print('============='+('\n'))
print('Shape of the dataset: {}\n'.format(train.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(train['text']))))
print('Total Number of missing values:\n{}\n\n'.format(train.isnull().sum()))
print('TEST DATA')
print('========='+('\n'))
print('Shape of the dataset: {}\n'.format(test.shape))
print('Total Number of unique tweets: {}\n'.format(len(set(test['text']))))
print('Total Number of missing values:\n{}\n' .format(test.isnull().sum()))

TRAINING DATA

Shape of the dataset: (33000, 2)

Total Number of unique tweets: 29948

Total Number of missing values:
lang_id    0
text       0
dtype: int64


TEST DATA

Shape of the dataset: (5682, 2)

Total Number of unique tweets: 5459

Total Number of missing values:
index    0
text     0
dtype: int64

time: 94 ms (started: 2022-06-25 22:28:07 +01:00)


## Data Preprocessing

In [7]:
def text_preprocessing(text):
    
    '''
    This functions cleans text from line breaks, URLs, numbers, etc.
    '''
    
    text = text.lower() #to lower case
    text = text.replace('\n', ' ') # remove line breaks
    text = text.replace('\@(\w*)', '') # remove mentions
    text = re.sub(r"\bhttps://t.co/\w+", '', text) # remove URLs
    text = re.sub('\w*\d\w*', '', text) # remove numbers
    text = re.sub(r'\#', '', text) # remove hashtags. To remove full hashtag: '\#(\w*)'
    text = re.sub('\w*\d\w*', '', text) # removes numbers?
    text = re.sub(' +', ' ', text) # remove 1+ spaces
    text = re.sub("\n"," ",text)
    text =' '.join(text.split())

    return text

time: 78 ms (started: 2022-06-25 22:28:07 +01:00)


In [8]:
# Splitting the labels and features
train.head(5)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


time: 125 ms (started: 2022-06-25 22:28:07 +01:00)


In [9]:
test.head(5)

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


time: 156 ms (started: 2022-06-25 22:28:07 +01:00)


In [10]:
train.text.duplicated(keep="first").value_counts()

False    29948
True      3052
Name: text, dtype: int64

time: 203 ms (started: 2022-06-25 22:28:07 +01:00)


In [11]:
test.text.duplicated(keep="first").value_counts()

False    5459
True      223
Name: text, dtype: int64

time: 157 ms (started: 2022-06-25 22:28:07 +01:00)


In [12]:
# Looking for duplicates
percent_duplicates = round((1-(train['text'].nunique()/len(train['text'])))*100,2)
print('Duplicated text in train data:')
print(percent_duplicates,'%')

Duplicated text in train data:
9.25 %
time: 282 ms (started: 2022-06-25 22:28:07 +01:00)


In [13]:
train.drop_duplicates(subset="text",keep="first",inplace=True,ignore_index=True)
train.shape

(29948, 2)

time: 187 ms (started: 2022-06-25 22:28:08 +01:00)


In [14]:
# Looking for duplicates
percent_duplicates = round((1-(test['text'].nunique()/len(test['text'])))*100,2)
print('Duplicated text in train data:')
print(percent_duplicates,'%')

Duplicated text in train data:
3.92 %
time: 125 ms (started: 2022-06-25 22:28:08 +01:00)


In [15]:
train['text'] = train['text'].apply(text_preprocessing)
test['text'] = test['text'].apply(text_preprocessing)

time: 6.16 s (started: 2022-06-25 22:28:08 +01:00)


In [16]:
# Replace '.txt' with 'text file'
train["text"] = train["text"].str.replace(".txt", " text file")
test["text"] = test["text"].str.replace(".txt", " text file")

C:\Users\SAMSON\AppData\Local\Temp/ipykernel_12304/2305434871.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train["text"] = train["text"].str.replace(".txt", " text file")


time: 359 ms (started: 2022-06-25 22:28:14 +01:00)


C:\Users\SAMSON\AppData\Local\Temp/ipykernel_12304/2305434871.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test["text"] = test["text"].str.replace(".txt", " text file")


Great! We have dealt with all the duplicates in our dataset.

In [17]:
# Removing extra spaces
train['text']=train['text'].apply(lambda x: re.sub(' +',' ',x))

time: 781 ms (started: 2022-06-25 22:28:15 +01:00)


In [18]:
# Removing extra spaces
test['text']=test['text'].apply(lambda x: re.sub(' +',' ',x))

time: 141 ms (started: 2022-06-25 22:28:15 +01:00)


###### Create a copy

The first step in the preprocessing is to create a copy of the train dataframe for the EDA.In this step we start by determining the length of each text and storing this information in a new column. We then tokenize the text 

In [22]:
 df = train.copy()

time: 16 ms (started: 2022-06-25 22:31:16 +01:00)


In [25]:
def lemma(df):
    wnl = WordNetLemmatizer()
    df['length'] = df['text'].str.len()
    df['tokenized'] = df['text'].apply(word_tokenize)
    return df

df = lemma(df)
df.head()  

,lang_id,text,length,tokenized
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,220,"[umgaqo-siseko, wenza, amalungiselelo, kumazik..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,252,"[i-dha, iya, kuba, nobulumko, bokubeka, umsebe..."
2,eng,the province of kwazulu-natal department of tr...,264,"[the, province, of, kwazulu-natal, department,..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,217,"[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,239,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


time: 12.6 s (started: 2022-06-25 22:32:09 +01:00)


In [ ]:
# Importing spacy
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['lemmatized']=df['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
df_grouped=df[['text','lemmatized']].groupby(by='text').agg(lambda x:' '.join(x))
df_grouped.head()

In [ ]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['lemmatized'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

## EXPLORATORY data Analysis

In [ ]:
def frequency(text):
    # Count vectorizer excluding english stopwords
    cv = CountVectorizer(stop_words='english')
    words = cv.fit_transform(text)
    
    # Count the words in the texts and determine the frequency of each word
    sum_words = words.sum(axis=0)
    words_freq = [(word, sum_words[0, i]) for word, i in cv.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    
    # Create a dataframe to store the top 25 words and their frequencies
    frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])
    frequency = frequency.head(25)
    
    return frequency

In [ ]:
frequency(df)

In [ ]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
#from wordcloud import WordCloud
#from textwrap import wrap

# Function for generating word clouds
#def generate_wordcloud(data,title):
# plt.figure(figsize=(10,8))
#  plt.imshow(wc, interpolation='bilinear')
 # plt.axis("off")
 # plt.title('\n'.join(wrap(title,60)),fontsize=13)
  #plt.show()
  # Transposing document term matrix
#df_dtm=df_dtm.transpose()

# Plotting word cloud for each product
#for index,product in enumerate(df_dtm.columns):
#generate_wordcloud(df_dtm[product].sort_values(ascending=False),product)
# please uncomment it was wasting time and I need to submit

In [ ]:
#vocab = set()
#corpus= [x.split() for x in df['text'].tolist()]
#for sentence in corpus:
  for word in sentence:
 #   vocab.add(word.lower())
#print("Number of distinct words in raw data: ", len(vocab))

## Feature Engineering

### Splitting out X (indepedent) and Y (target/dependent) variables

In [ ]:
X = train['text']
y = train['lang_id']

### Splitting of Training and Validation Sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

### Model Building

In [ ]:

classifiers = [LinearSVC(random_state=42),
                SVC(),
                #DecisionTreeClassifier(),
                #RandomForestClassifier(n_estimators=100, max_depth=2,
                                     #random_state=0, class_weight="balanced"),
                #MLPClassifier(alpha=1e-5,
                             #hidden_layer_sizes=(5, 2),
                             #random_state=42),
               LogisticRegression(random_state=42,
                                  multi_class='ovr',
                                  n_jobs=1,
                                  C=1e5,
                                  max_iter=100000),
               KNeighborsClassifier(n_neighbors=5),
               MultinomialNB(),
               ComplementNB(),
               SGDClassifier(loss='hinge',
                             penalty='l2',
                             alpha=1e-3,
                             random_state=42,
                             max_iter=5,
                             tol=None)
                #GradientBoostingClassifier(),
                #xgb.XGBClassifier(learning_rate=0.1,
                #                 n_estimators=1000,
                #                 max_depth=5,
                #                min_child_weight=1,
                #                 gamma=0,
                #                subsample=0.8,
                #                 colsample_bytree=0.8,
                #                 nthread=4,
                #                 seed=27)
               ]
# The commented code took tool long to load hence the reason for excluding them also, they didn't really perform as much as the ran

### Creating Function for Model Building

In [ ]:
def models_building(classifiers, X_train, y_train, X_val, y_val):
    """
    This function takes in a list of classifiers
    and both the train and validation sets
    and return a summary of F1-score and
    processing time as a dataframe

    Input:
    classifiers: a list of classifiers to train
                 datatype: list
    X_train: independent variable for training
             datatype: series
    y_train: dependent variable for training
             datatype: series
    X_val: independent variable for validation
           datatype: series
    y_val: dependent variable for validation
           datatype: series

    Output:
    model_summary: F1 Score for all the classifiers
                   datatype: dataframe
    """

    models_summary = {}

    # Pipeline to balance the classses and then to build the model
    for clf in classifiers:
        clf_text = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                       max_df=0.9,
                                                       ngram_range=(1, 2))),
                             ('clf', clf)])

        # Logging the Execution Time for each model
        start_time = time.time()
        clf_text.fit(X_train, y_train)
        predictions = clf_text.predict(X_val)
        run_time = time.time()-start_time

        # Output for each model
        models_summary[clf.__class__.__name__] = {
            'F1-Macro': metrics.f1_score(y_val,
                                         predictions,
                                         average='macro'),
            'F1-Accuracy': metrics.f1_score(y_val, predictions,
                                            average='micro'),
            'F1-Weighted': metrics.f1_score(y_val,
                                            predictions,
                                            average='weighted'),
            'Execution Time': run_time}

    return pd.DataFrame.from_dict(models_summary, orient='index')

### Execution of the Classifiers

In [ ]:
import time
classifiers_df = models_building(classifiers, X_train, y_train, X_val, y_val)
ordered_df = classifiers_df.sort_values('F1-Macro', ascending=False)
ordered_df

### Comparing Classification Methods
The most performing is the Multinomial Naive Bayes with F1-Macro of 99.9% and accuracy of 99.9% while closely followed by Complement Naive Bayes, Logistic Regression, Linear Support Vector Classifier, Support Vector Machine etc.

Hyperameter tuning is carried out to improve the F1-Macro score

### Hyperparameter Tuning on Most Performing Models

In [ ]:
# Refining the train-test split for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01)

In [ ]:
### Multinomial Naive Bayes

In [ ]:
from sklearn.model_selection import StratifiedKFold
# Creating a pipeline for the gridsearch
param_grid = {'alpha': [0.1, 1, 5, 10]}  # setting parameter grid

tuned_mnb = Pipeline([('tfidf', TfidfVectorizer(min_df=1,
                                                max_df=0.8,
                                                ngram_range=(1, 2))),
                      ('mnb', GridSearchCV(MultinomialNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           n_jobs=-1,
                                           scoring='f1_weighted'))
                      ])

tuned_mnb.fit(X_train, y_train)  # Fitting the model

y_pred_mnb = tuned_mnb.predict(X_val)  # predicting the fit on validation set

print(classification_report(y_val, y_pred_mnb))


### Creating File for Submission

In [ ]:
submission_trial = pd.DataFrame(test['index'])
submission_trial['lang_id'] = tuned_mnb.predict(test['text'])
submission_trial.to_csv('submission_mn.csv', index=False)

## Conclusion
Several algorithms were tried and Multinomial Naive Bayes classifier was the most performing. It performed very well on the training and validation datasets with an accuracy score of over 99% and F1 Macro score of over 99%. After testing the fitted model on the held-out/unseen dataset, it was able to predict the classes of languages with an F1 Score of about 98%.

### References
https://colab.research.google.com/drive/1eQydhzxK03z4_20_A9dunOq04trRYuZG?usp=sharing#scrollTo=InfhMWcIHlR_
https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/
https://stackoverflow.com/questions/46864696/how-to-resolve-python-error-while-generating-pie-chart-valueerror-explode-m